# RA2CE feature: Uncertainty : Hazard Overlay 

In [ ]:
import numpy as np
from pathlib import Path
import geopandas as gpd
import folium 
from shapely.geometry import LineString, Polygon, box
import rasterio
import matplotlib.pyplot as plt

In [ ]:
import ra2ce.network.networks_utils as nut
from ra2ce.network.network_config_data.enums.network_type_enum import NetworkTypeEnum
from ra2ce.network.network_config_data.enums.road_type_enum import RoadTypeEnum

In [ ]:
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.network.network_config_data.network_config_data import (NetworkConfigData,NetworkSection,HazardSection,)
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.network_config_data import (NetworkSection)

In [ ]:
# specify the name of the path to the project folder where you created the RA2CE folder setup
root_dir = Path(r'C:\Users\gunaratn\RA2CE\Uncertainty\Test2') #Set your own path!!
assert root_dir.exists()

hazard_folder = root_dir / "static" / "hazard" # find the hazard folder where you locate your hazard map # set the name of the hazard map
hazard_map = Path(list(hazard_folder.glob("*.tif"))[0])
hazard_map

In [ ]:
# Open the Hazard map file using rasterio
with rasterio.open(hazard_map) as src:
    tif_array = src.read(1)  
    src_crs = src.crs
    print("Hazard Map is in a the CRS:", src_crs)

plt.figure(figsize=(10, 10))
plt.imshow(tif_array, cmap='Blues')  
plt.colorbar(label='Pixel Values')
plt.title('hazard map')
plt.show()  

## Now make a configuration of the network and analyses parameters

In [ ]:
from ra2ce.ra2ce_handler import Ra2ceHandler

from ra2ce.analysis.damages.damages import AnalysisSectionDamages
from ra2ce.configuration.ra2ce_enum_base import Ra2ceEnumBase
from ra2ce.analysis.analysis_config_data.enums.analysis_damages_enum import AnalysisDamagesEnum
from ra2ce.analysis.analysis_config_data.enums.event_type_enum import EventTypeEnum
from ra2ce.analysis.analysis_config_data.enums.damage_curve_enum import DamageCurveEnum
from ra2ce.analysis.analysis_config_data.analysis_config_data import AnalysisConfigData

In [ ]:
static_path=root_dir.joinpath("static")
output_path=static_path.joinpath("output_graph")

output_path

In [ ]:
#First we define which roads we will want to download from OSM to create a network with
_network_section = NetworkSection(
    source= SourceEnum.SHAPEFILE,       #Used to specify the shapefile name of the (road) network to do the analysis with, when creating a network from a shapefile.
    primary_file = [root_dir/"static"/"network"/"edges_NISv_RD_new_LinkNr.shp"], #soecify in the RA2CE folder setup where the network is locates
    save_gpkg=True
)

_hazard = HazardSection(
    hazard_map=[hazard_map],    #[Path(geotiff_files[0])],
    hazard_field_name= ['waterdepth'],
    aggregate_wl = AggregateWlEnum.MAX,
    hazard_crs = "EPSG:28992"
    #hazard_crs = 'EPSG:4326' #"EPSG:32736"
)

#pass the specified sections as arguments for configuration
_network_config_data = NetworkConfigData(
    root_path=root_dir,
    static_path=static_path,
    output_path=output_path,
    network=_network_section,
    hazard=_hazard
    )


Run RA2CE.

In [ ]:
handler = Ra2ceHandler.from_config(_network_config_data, None)

handler.configure()
handler.run_analysis()

#### Visualising the results

In [ ]:
analysis_output_path = static_path / "output_graph" 
hazard_gdf = gpd.read_file(analysis_output_path / 'base_graph_hazard_edges.gpkg')
hazard_gdf.head() 

In [ ]:
hazard_gdf.explore(column="EV1_ma", cmap = "Reds")